<a href="https://colab.research.google.com/github/kaditi14/LP-I-/blob/main/final_HPC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
#installing nvcc jupyter extension

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-xbq4jusp
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-xbq4jusp
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=53b82eed1e32a5fcf278861bfb792ce1a64842ce8518a73e6943c9accf40ca6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-evat3tq9/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
#load extension into environment
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu

#include<iostream>
#include<cstdio>
#include<cstdlib>
#include<cuda_runtime.h>
#include<iomanip>
using namespace std;

__global__ void minimum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] < input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void maximum(int *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads>0) {
        if(tid < number_of_threads) {
            int first = tid*step_size*2;
            int second = first + step_size;
            if(input[second] > input[first])
              input[first] = input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
    }
}

__global__ void sum(int *input) {
    const int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

__global__ void mean_diff_sq(float *input, float mean) {
    input[threadIdx.x] -= mean;
    input[threadIdx.x] *= input[threadIdx.x];
}

__global__ void sum_floats(float *input) {
    int tid = threadIdx.x;
    int step_size = 1;
    int number_of_threads = blockDim.x;
    
    while(number_of_threads > 0) {
        if(tid < number_of_threads) {
            int first = tid * step_size * 2;
            int second = first + step_size;
            
            input[first] += input[second];
        }
        step_size <<= 1;
        number_of_threads >>= 1;
       
    }
}

void copy_int_to_float(float *dest, int *src, int size){
    for(int i=0; i<size; i++)
        dest[i] = float(src[i]);
}

int main() {
    int n=20;
    srand(n);
	  int *arr=new int[n];
    int min=20000;
    //# Generate Input array using rand()
	  for(int i=0;i<n;i++)
	  {
		  arr[i]=rand()%20000;
      if(arr[i]<min)
        min=arr[i];
      cout<<arr[i]<<" ";
	  }
    cout<<endl;
    int size = n*sizeof(int); //calculate no. of bytes for array
    int *arr_d, result;
    
    cudaMalloc((void **)&arr_d, size);
    
    //MIN
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    minimum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The minimum element is "<<result<<endl;
      
       
    //MAX
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    maximum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The maximum element is "<<result<<endl;
    
    //SUM
    cudaMemcpy(arr_d, arr, size, cudaMemcpyHostToDevice);
    
    sum<<<1,n/2>>>(arr_d);
    
    cudaMemcpy(&result, arr_d, sizeof(int), cudaMemcpyDeviceToHost);
    
    cout<<"The sum is "<<result<<endl;
    
    //AVERAGE
    
    float mean = float(result)/n;
    cout<<"The mean is "<<mean<<endl;
    
    //STANDARD DEVIATION
    float *arr_float;
    float *arr_std, stdValue;
    
    arr_float = (float *)malloc(n*sizeof(float));
    cudaMalloc((void **)&arr_std, n*sizeof(float));
    
    copy_int_to_float(arr_float, arr, n);
    
    cudaMemcpy(arr_std, arr_float, n*sizeof(float), cudaMemcpyHostToDevice);
    
    mean_diff_sq <<<1,n>>>(arr_std, mean);
    sum_floats<<<1,n/2>>>(arr_std);
    
    cudaMemcpy(&stdValue, arr_std, sizeof(float), cudaMemcpyDeviceToHost);
    
    
    stdValue = stdValue / n;
    std::cout << std::fixed << std::setprecision(2)<<"The variance is "<< stdValue << std::endl;
    stdValue = sqrt(stdValue);
    
    cout<<"The standard deviation is "<<stdValue<<endl;
    
    cudaFree(arr_d);
           
    return 0;
}

11911 14258 18497 13959 13496 8381 13262 16441 6440 12481 18350 5724 13847 18409 10744 17409 10765 15607 9379 13859 
The minimum element is 5724
The maximum element is 18497
The sum is 213609
The mean is 10680.5
The variance is 17943516.00
The standard deviation is 4235.98

